In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch

In [3]:
#consider input sequence of "Your journey starts with one step"

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1-input x1)
   [0.55, 0.87, 0.66], # journey  (x^2-input x2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
#each row is a separete token and each row has a 3 dimensional vector for each vector


In [ ]:
#calculating context vector according to 2nd input(journey)
#context vector tells how each words realted with query token
query=inputs[1]#2nd token

attn_scores_2=torch.empty(inputs.shape[0])#creating a empty vector for context vector for 2nd token
for i,x_i in enumerate(inputs):
    attn_scores_2[i]=torch.dot(x_i,query)#each input token embedding's dot product with query token's vector embedding

print(attn_scores_2)#context vector for 2nd word


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
#normalisation is required for interpretability(tells how much each word is important so we want it to sum up to 1 )
attn_scores_2_tmp=attn_scores_2/attn_scores_2.sum()

print("Attention weights:",attn_scores_2_tmp)
print("Sum: ",attn_scores_2_tmp.sum())
#tells how much percentage each word is important to the query token and sum of all the values sum up to 1


Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum:  tensor(1.0000)


In [7]:
#use softmax activation-divides the overall probability to 1

def softmax_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)#dim=0 because we are summing all row value's exponential values in denominator

attn_scores_2_naive=softmax_naive(attn_scores_2)
print("Attention weights:",attn_scores_2_naive)
print("Sum: ",attn_scores_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [8]:
#use inbuilt pytorch softmax to prevent overflow and underflow
attn_weights_2=torch.softmax(attn_scores_2,dim=0)
print("Attention weights:",attn_weights_2)
print("Sum: ",attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum:  tensor(1.)


In [ ]:
#calculating context vector for token "journey"
query=inputs[1]

context_vec_2=torch.zeros(query.shape)

for i,x_i in enumerate(inputs):
    context_vec_2+=attn_weights_2[i]*x_i #scalling down by multiplying vector embedding(x_i) with attention wieght and each vector embedding has 3 dimensions and summing all of the sacled down resultant vector


print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [ ]:
#attention matrix-each row contains attentions scores for each input token

attn_scores=torch.empty(6,6)

for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        attn_scores[i,j]=torch.dot(x_i,x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
#for loops are very slow-so use linear algebra tricks
#step1:finding dot product between each query token and all input tokens
attn_scores=inputs@inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
#step 2-normalizing attention scores
attn_weights=torch.softmax(attn_scores,dim=-1)#-1 tells to work with last dimension , here it is 2d matrix(6,6)last dimension is column,normalizing along columns of each row
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [15]:
#step-3:computing context vector
all_context_vectors=attn_weights@inputs
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
